In [11]:
from create_model import *
#importing functions from .py file

In [3]:
directory = "/Users/justycewatson/Desktop/CIERA_files/NGC2682/jw_output"
#computer directory the files will come from

model_cluster_statistic = create_stats(directory)
# creating the data table using 'create_stats' function with the directory files
 
model_cluster_statistic

,source_id,Width,Upper_bound,Lower_bound,Stdev,SnR,Dip_p,Dip_value,KS_value,KS_p,ESS
0,605002016872204416,1.361236,0.557799,0.967022,0.658567,14.214572,0.000000,0.017193,0.213815,3.038420e-250,9475.632108
1,604942879467199360,1.352529,0.557799,0.967022,0.610061,15.058561,0.000000,0.019429,0.166502,1.142975e-147,10231.678292
2,604969031523465728,0.387865,0.557799,0.967022,0.263017,36.229304,0.000000,0.012712,0.148434,3.486940e-120,9596.183794
3,604921679508385664,1.676418,0.557799,0.967022,0.744099,12.044454,0.000001,0.008560,0.136923,1.933852e-101,9597.816566
4,604968958508607360,1.262185,0.557799,0.967022,0.600071,15.298958,0.000000,0.014429,0.142905,7.963583e-108,9969.498500
...,...,...,...,...,...,...,...,...,...,...,...
1423,604994045412975744,0.770949,0.557799,0.967022,0.473367,19.784625,0.000000,0.009089,0.167184,4.178415e-150,9861.275938
1424,604921924322164608,0.167193,0.557799,0.967022,0.091203,105.613775,0.000000,0.012850,0.154449,4.685580e-128,10044.534580
1425,604970062315630336,1.360707,0.557799,0.967022,0.639646,14.251756,0.000000,0.013919,0.147220,1.930928e-114,9815.960047
1426,598962292125778560,1.532101,0.557799,0.967022,0.686447,13.055923,0.000000,0.020750,0.124527,1.440514e-82,9909.201340


# Creating ML model

In [4]:
##reading in dataframe containing known sampling quality to train model on

df1 = pd.read_csv('/Users/justycewatson/Desktop/CIERA_files/NGC2682/CSV_Files/NGC2682_Age_Stats.csv',sep=',')
sampling_df = df1[df1['Single Sampling'].isna() == False]

In [5]:
X, y, clf, X_test, X_train, y_test = create_model(model_cluster_statistic, sampling_df)
y_pred = make_preds(X_test, clf, y_test = y_test)

Accuracy: 0.969626168224299
              precision    recall  f1-score   support

         Bad       0.99      0.97      0.98       359
        Good       0.88      0.94      0.91        69

    accuracy                           0.97       428
   macro avg       0.93      0.96      0.95       428
weighted avg       0.97      0.97      0.97       428

Feature Importance Ranking:
Width          0.347775
SnR            0.213674
Stdev          0.169446
KS_value       0.159941
Dip_value      0.071619
ESS            0.027710
Dip_p          0.008100
KS_p           0.001736
Upper_bound    0.000000
Lower_bound    0.000000
dtype: float64


/Users/justycewatson/Desktop/CIERA_files/BASE9_ML/create_model.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampling_df['source_id'] = sampling_df['source_id'].astype(str)


In [24]:
save_model(clf)